# First object condensation model training

* **Requirements*: You need to have graphs constructed, e.g., with `010_build_graphs.ipynb`

In [1]:
from pathlib import Path

from gnn_tracking.postprocessing.dbscanscanner import dbscan_scan

from gnn_tracking.models.track_condensation_networks import GraphTCN
from gnn_tracking.training.tcn_trainer import TCNTrainer
from gnn_tracking.metrics.losses import (
    EdgeWeightFocalLoss,
    PotentialLoss,
    BackgroundLoss,
)
from gnn_tracking.utils.loading import get_loaders, TrackingDataset

In [2]:
graph_dir = Path.home() / "data" / "gnn_tracking" / "graphs"
assert graph_dir.is_dir()

In [3]:
datasets = {
    "train": TrackingDataset(graph_dir, stop=9),
    "val": TrackingDataset(graph_dir, start=9, stop=10),
}
loaders = get_loaders(datasets)

[17:28:57] INFO: DataLoader will load 9 graphs (out of 10 available).
[17:28:57] DEBUG: First graph is /Users/fuchur/data/gnn_tracking/graphs/data21000_s5.pt, last graph is /Users/fuchur/data/gnn_tracking/graphs/data21008_s10.pt
[17:28:57] INFO: DataLoader will load 1 graphs (out of 10 available).
[17:28:57] DEBUG: First graph is /Users/fuchur/data/gnn_tracking/graphs/data21008_s24.pt, last graph is /Users/fuchur/data/gnn_tracking/graphs/data21008_s24.pt
[17:28:57] DEBUG: Parameters for data loader 'train': {'batch_size': 1, 'num_workers': 1, 'sampler': <torch.utils.data.sampler.RandomSampler object at 0x11154c4f0>, 'pin_memory': True}
[17:28:57] DEBUG: Parameters for data loader 'val': {'batch_size': 1, 'num_workers': 1, 'sampler': None, 'pin_memory': True}


In [4]:
loss_functions = {
    "edge": (EdgeWeightFocalLoss(gamma=5, alpha=0.95), 500.0),
    "potential": (PotentialLoss(q_min=0.01), {"attractive": 500, "repulsive": 5}),
    "background": (BackgroundLoss(sb=1), 0.05),
}

In [8]:
model = GraphTCN(
    node_indim=datasets["train"].num_node_features,
    edge_indim=datasets["train"].num_edge_features,
    h_dim=10,
    e_dim=10,
    L_ec=5,
    L_hc=2,
    h_outdim=10,
    hidden_dim=128,
)

In [9]:
clustering_functions = {"dbscan": dbscan_scan}

In [10]:
trainer = TCNTrainer(
    model=model,
    loaders=loaders,
    loss_functions=loss_functions,
    lr=0.005,
    cluster_functions=clustering_functions,
)

[17:30:33 TCNTrainer] INFO: Using device cpu


In [11]:
trainer.train(epochs=2)

[17:30:37 TCNTrainer] INFO: Epoch  1 (    0/9): edge_weighted=   0.91777, potential_attractive_weighted=   0.00905, potential_repulsive_weighted=1862.03873, background_weighted=   0.04993
[17:30:46 TCNTrainer] INFO: Training for epoch 1 took 10.92 seconds
/Users/fuchur/.miniconda3/envs/gnn/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/fuchur/.miniconda3/envs/gnn/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3747: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/fuchur/.miniconda3/envs/gnn/lib/python3.9/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
[17:30:53 ClusterHP] WARNING: Only using 1 thread for DBSCAN scan
[17:30:53 ClusterHP] DEBUG: Starting from params: {'eps': 0.95, 'min